In [12]:
#cnn
"""data_load関数
   X_train,y_train=load_data('train')
   X_test,y_test=load_data('test)
   cnn:tensorflow layersAPiで実装"""

import os
import numpy as np
import tensorflow as tf
import random
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

label_dict={'aoyama':0,'komazawa':1,'meigaku':2,'misaki':3,'obirin':4,'phoenix':5,'saidai':6
           ,'sekei':7,'senshu':8,'teikyo':9,'tokyo':10,'twu':11,'ycu':12,'ynu':13}

data_type='train'
#filename,image,labelを格納するリストを作成
#画像とラベルを返す
filenames=[]
images=[]
labels=[]
    
    
#条件に一致する要素のみを抽出
#faces directoryからdirectoryが何もない以外のものとtrain dataのみを抽出
walk=filter(lambda x:not len(x[1]),os.walk(r'D:/faces/{}/'.format(data_type.strip())) )
    
    
for root,dirs,files in walk:
    filenames+=['{}/{}'.format(root,path) for path in files if not path.startswith('.')]
#ランダムなファイルネームを選択
random.shuffle(filenames)
images=[]
for idx in range(len(filenames)):
    file=filenames[idx]
    image=cv2.imread(file,3)
    image=cv2.resize(image,(32,32))
    image=image.reshape((-1,32*32))
    
    images.append(image)

#labelを作成
for filename in filenames:
    label=np.zeros(14)
    filename=filename.split('/')[-2]
    filename=filename[:-4]

    for key,value in label_dict.items():
        if key in filename:
            label[value]=1
    labels.append(list(label))
    
print(np.array(images).shape)
print(np.array(labels).shape)

X_train=np.array(images).reshape(-1,32,32,3)/255
X_train=X_train.astype('float32')
y_train=np.array(labels)




(867, 3, 1024)
(867, 14)
(867, 32, 32, 3)


In [26]:
import os
import numpy as np
import cv2
import random
def load_data(data_type):
    filenames=[]
    images=[]#x_
    labels=[]#y_
    
    #faces directoryからdata_typeを取り出す
    walk=filter(lambda x: not len(x[1]),os.walk(r'D:/faces/{}'.format(data_type.strip())))
    
    for root,dirs,files in walk:
        filenames+=['{}/{}'.format(root,path) for path in files if not path.startswith('.')]
        
        
    #filenamesをランダムに選択
    random.shuffle(filenames)
    for idx in range(len(filenames)):
        image=cv2.imread(filenames[idx],3)#chunnel3 RGB
        image=cv2.resize(image,(32,32))
        image=image.reshape((-1,32*32))
        
        images.append(image)
        
    for filename in filenames:
        #labelの初期定義
        label=np.zeros(14)
        filename=filename.split('/')[-2]
        filename=filename[:-4]
        #辞書形式をイタレーション
        for key,value in label_dict.items():
            if key in filename:
                label[value]=1
        labels.append(list(label))
        
    images=np.array(images)
    labels=np.array(labels)
    return images,labels

X_train,y_train=load_data('train')
X_test,y_test=load_data('test')

X_train=X_train.reshape(-1,32,32,3).astype('float32')/255
X_test=X_test.reshape(-1,32,32,3).astype('float32')/255

print('train_shapoe:',X_train.shape)
print('test_shape:',X_test.shape)


train_shapoe: (867, 32, 32, 3)
test_shape: (228, 32, 32, 3)


In [27]:
#cnn.py

"""batch学習をする時点でX_dataは一次元配列に変換されている"""
def batch_generator(X_train,y_train,batch_size=25,shuffle=True,random_state=None):
    X_data=np.array(X_train)
    y_data=np.array(y_train)
    idx=np.arange(y_data.shape[0])
    np.random.RandomState(random_state).shuffle(idx)
    
    X_data=X_data[idx]
    y_data=y_data[idx]
    
    for i in range(0,X_data.shape[0],batch_size):
        yield(X_data[i:i+batch_size,:],y_data[i:i+batch_size])


class CNN(object):
    """init:batch_size バッチ学習、epochs:学習回数,learning_rate:学習率、drop_out:ドロップアウト率
       shuffle:シャッフルするかどうか、shuffleするのでseedを固定
       build:
       train:
       load:
       save:
       predict:"""
    
    def __init__(self,batch_size=25,epochs=20,learning_rate=1.0e-4,drop_out=0.5,shuffle=True,
                random_state=None):
        np.random.seed(random_state)
        
        self.batch_size=batch_size
        self.epochs=epochs
        self.learning_rate=learning_rate
        self.drop_out=drop_out
        self.shuffle=shuffle
        
        #tensorflowのGraphの初期化
        g=tf.Graph()
        with g.as_default():
            tf.set_random_seed(random_state)
            
            
            #CNN構築
            self.build()
            
            #変数を初期化
            self.init_op=tf.global_variables_initializer()
            
            #saverの設定
            self.saver=tf.train.Saver()
            
        self.sess=tf.Session(graph=g)
        
    def build(self):
        tf_x=tf.placeholder(tf.float32,shape=[None,32*32*3],name='tf_x')
        tf_y=tf.placeholder(tf.int32,shape=[None,14],name='tf_y')
        is_train=tf.placeholder(tf.bool,shape=(),name='is_train')
        
        #xを四次元テンソルに変換[batch_size,h,w,channels]
        tf_x_img=tf.reshape(tf_x,[-1,32,32,3],name='tf_x_img')
    
        #第一層
        h1=tf.layers.conv2d(tf_x_img,kernel_size=(5,5),filters=32,activation=tf.nn.relu)
        
        #pooling
        h1_pool=tf.layers.max_pooling2d(h1,pool_size=(2,2),strides=(2,2))
        
        #第二層
        h2=tf.layers.conv2d(h1_pool,kernel_size=(5,5),filters=64,activation=tf.nn.relu)
        
        #pooling
        h2_pool=tf.layers.max_pooling2d(h2,pool_size=(2,2),strides=(2,2))
        
        #第三層 全結合層
        #全結合層に前の形を取得
        input_shape=h2_pool.get_shape().as_list()
        #最初のバイアス以外をprod
        n_input_units=np.prod(input_shape[1:])
        h2_pool_flat=tf.reshape(h2_pool,shape=[-1,n_input_units])
        
        
        #全結合層はdenseを用いる
        h3=tf.layers.dense(h2_pool_flat,1024,activation=tf.nn.relu)
        
        #dropout
        h3_dropout=tf.layers.dropout(h3,rate=self.drop_out,training=is_train)
        
        #線形活性化　第四層
        h4=tf.layers.dense(h3_dropout,14,activation=None)
        
        #予測辞書
        predictions={'probabilities':tf.nn.softmax(h4,name='probabilities'),
                    'labels':tf.cast(tf.argmax(h4,axis=1),tf.int32,name='labels')}
        
        #損失関数
        cross_entropy_loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=h4,labels=tf_y),name='cross_entropy_loss')
        
        #最適化
        optimizer=tf.train.AdamOptimizer(self.learning_rate)
        optimizer=optimizer.minimize(cross_entropy_loss,name='train_op')
        
        #予測正解率を特定
        correct_predictions=tf.equal(predictions['labels'],tf_y,name='correct_preds')
        
        accuracy=tf.reduce_mean(tf.cast(correct_predictions,tf.float32),name='accuracy')
        
    #学習したモデルを保存する
    def save(self,epoch,path='./tflayers-model/'):
        if not os.path.isdir(path):
            os.mkdir(path)
        print('saving_model in {}'.format(path))
        self.saver.save(self.sess,os.path.join(path,'model.ckpt'),global_step=epoch)
        
    def load(self,epoch,path):
        print('loading model from {}'.format(path))
        
        self.saver.restore(self.sess,os.path.join(path,'model.ckpt-{}'.format(epoch)))
        
    def train(self,training_set,validation_set=None,initialize=True):
        if initialize:
            self.sess.run(self.init_op)
            
        X_data=np.array(training_set[0])
        y_data=np.array(training_set[1])
        
        #コスト関数を格納するリスト
        self.training_loss=[]
        
        for epoch in range(1,self.epochs+1):
            #バッチ学習のためのデータ作成
            batch_gen=batch_generator(X_data,y_data,shuffle=self.shuffle)
            #バッチ学習一回当たりの損失マン
            avg_loss=0.0
            for i,(batch_x,batch_y) in enumerate(batch_gen):
                feed={'tf_x_img:0':batch_x,'tf_y:0':batch_y,'is_train:0':True}#dropout
                
                loss,_=self.sess.run(['cross_entropy_loss:0','train_op'],feed_dict=feed)
                
                avg_loss+=loss
                
            print('epoch %02d: training loss: %7.3f'%(epoch,avg_loss))
            
            if validation_set is not None:
                feed={'tf_x_img:0':batch_x,'tf_y:0':batch_y,'is_train':False}
                
                valid_acc=self.sess.run('accuracy:0',feed_dict=feed)
                
                print('validation acc%7.3f'%valid_acc)
                
            else:
                print()
                
    
    def predict(self,X_test,return_proba=False):
        feed={'tf_x_img:0':X_test,'is_train:0':False}#dropoutなし
        if return_proba:
            self.sess.run('probabilities:0',feed_dict=feed)
            
        else:
            self.sess.run('labels:0',feed_dict=feed)
            
            
    
    

In [ ]:
#main関数
def main():
    
    #training phase
    cnn=CNN(random_state=123)
    cnn.train(training_set=(X_train,y_train))
    cnn.save(epoch=20)
    
    #test phase
    del cnn
    cnn2=CNN(random_state=123)
    cnn2.load(epoch=20,path='./tflayers-model')
    print('predictions\n',cnn2.predict(X_test[:10,:]))
    
    preds=cnn2.predict(X_test)
    print('test accuracy%7.3f'%(np.sum(preds=y_test)/len(preds)))
    
    
if __name__=='__main__':
    main()
    
    
    

epoch 01: training loss:  84.971

epoch 02: training loss:  83.386

epoch 03: training loss:  82.641

epoch 04: training loss:  82.145

epoch 05: training loss:  82.020

epoch 06: training loss:  80.863

epoch 07: training loss:  79.600

epoch 08: training loss:  79.192

epoch 09: training loss:  78.748

epoch 10: training loss:  77.702

epoch 11: training loss:  76.900

epoch 12: training loss:  75.405

epoch 13: training loss:  75.334

epoch 14: training loss:  74.820

epoch 15: training loss:  73.999

epoch 16: training loss:  72.418

epoch 17: training loss:  73.132

epoch 18: training loss:  71.658

epoch 19: training loss:  71.471

epoch 20: training loss:  70.672

saving_model in ./tflayers-model/
